# CL Protocol Specification

Code is used to show the mathematics details for ZKP-CL.

The CL-signature scheme defines the public key in this key pair as the quadruple $(n, Z, S ,\{R_i\}_{i \in A})$ where $A$ is the set of indices of attributes in the credential schema, $n$ is a Special RSA Modulus, and $Z, S , {R_i}$ are random quadratic residues modulo n.

The public key information $(n, Z, S ,\{R_i\}_{i \in A})$ are publicly available on the ledger, the signature $(A, e, v)$ can only be calculated by the issuer, who owns the private key $(p, q)$.

## Equations


Remember from the math section:

$$ T = A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $$
$$ A' = AS^r $$
$$ v' = v - er $$
$$ \hat{v} = \bar{v} + Dv'  $$
$$ e' = e - \beta $$
$$ \hat{e} = \bar{e} + De' $$
$$ \hat{m_i} = \bar{m_i} + Dm_i $$
$$ Q = A^e \pmod{n} $$
$$ Q = \Bigg(\frac{Z}{S^v \prod_{i \in A}{R_i^{m_i}}}\Bigg) \pmod{n} $$
$$ \hat{T} = \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n} $$

## Reference

- https://docs.google.com/presentation/d/14SkMM_LtC0YixOryO5GfMbdQiwps6b-vq-k1A-ZVRmk/edit#slide=id.g6f917b6d41_6_150
- https://github.com/WebOfTrustInfo/rwot5-boston/blob/master/final-documents/data-minimization-sd.md#crypto-details
- https://uia.brage.unit.no/uia-xmlui/bitstream/handle/11250/2593825/PhD%2BDissertation_G.%2BP.%2BHARSHA%2BSANDARUWAN_FINAL.pdf#page=64
- https://github.com/PeterAltmann/SSIdemo/blob/main/CEF_final_report.ipynb
- https://www.cs.ru.nl/~gergely/objects/ZK-AnonCred.pdf#page=25
- http://www.cs.ru.nl/~gergely/objects/u-prove.pdf
- https://eprint.iacr.org/2022/492.pdf#page=7
- https://eprint.iacr.org/2010/496.pdf#page=24
- https://github.com/hyperledger-archives/indy-anoncreds/blob/master/docs/dev/anoncred.tex
- https://hyperledger-indy.readthedocs.io/projects/hipe/en/latest/text/0109-anoncreds-protocol/README.html#issuance-of-credentials

## Issuer setup

Let $i$ be the number of attributes we work with.

Steps to set up the issuer

1. Issuer chooses a number of attribute bases $i$
2. Generate random primes of equal length ($p$ and $q$) 
3. Then compute the modulus $n \leftarrow pq$.
4. Generate a random quadratic residue: $S \pmod{n}$
5. Generate quadratic residues $(Z,\{R_i\}_{i \in A}) \pmod{n}$ 

The issuer's public key is $(n, S, Z, R_1,R_2,\ldots,R_i)$ and the private key is $(p, q)$.

In [41]:
from Crypto.Util import number
import hashlib


In [42]:
# For demo purposes to make print more readable and consistent
bitsize = 32
p = 3397833353
q = 2516557159
# p = number.getPrime(bitsize)
# q = number.getPrime(bitsize)

# The public parts
n = p*q
nsize = 2*bitsize
# Generate quadratic residues mod n
R_1 = pow(number.getRandomRange(2, n), 2, n)
R_2 = pow(number.getRandomRange(2, n), 2, n)
R_3 = pow(number.getRandomRange(2, n), 2, n)
R_4 = pow(number.getRandomRange(2, n), 2, n)
R_5 = pow(number.getRandomRange(2, n), 2, n)
R_6 = pow(number.getRandomRange(2, n), 2, n)
S = pow(number.getRandomRange(2, n), 2, n)
Z = pow(number.getRandomRange(2, n), 2, n)


print(f'Public key is:\nn = {n}\n(R_1, R_2, R_3, R_4, R_5, R_6) = {(R_1, R_2, R_3, R_4, R_5, R_6)}\nS = {S}\nZ = {Z}')
print(f'\nPrivate key is: {(p,q)}')

Public key is:
n = 8550841849581124127
(R_1, R_2, R_3, R_4, R_5, R_6) = (133561300992591870, 5250103334166041682, 6301733490984978747, 7776765663839157772, 2507362099940342639, 5432598073234467476)
S = 1430320840990158594
Z = 8407419917135409481

Private key is: (3397833353, 2516557159)


In [43]:
# The messages
msize = nsize
m_1 = number.getRandomRange(2, n)
m_2 = number.getRandomRange(2, n)
m_3 = number.getRandomRange(2, n)
m_4 = number.getRandomRange(2, n)
m_5 = number.getRandomRange(2, n)
m_6 = number.getRandomRange(2, n)

print(f'Message is:\nm_1, m_2, m_3, m_4, m_5, m_6)  = {(m_1, m_2, m_3, m_4, m_5, m_6)}')

Message is:
m_1, m_2, m_3, m_4, m_5, m_6)  = (2361185764792719478, 3838706582981664516, 3142452892453462317, 1094459997095202099, 3041891509396781501, 4962249658687791127)


## User

User loads Issuer's public key and calculate:

1. Generates random integer $v'$
2. Then computes
$$U \leftarrow S^{v'}R_1^{m_1}\pmod{n}$$

In [44]:
# Generate random integear
v_prim = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate U
U = pow(S, v_prim, n) * pow(R_1, m_1, n)

### User send to Issuer
User send $(U, v')$ to the Issuer

## Issuer Signature Generation


Issuer generate CL signature on the attributes by computing:

1. Generates random integer $v''$
2. Generates random prime $e$
3. Then computes 
$$
Q \leftarrow \frac{Z}{U S^{v''}(R_2^{m_2}R_3^{m_3}\cdots  R_i^{m_i})}\pmod{n}.
$$
4. Then computes
$$
A \leftarrow Q^{e^{-1}\pmod{p'q'}}\pmod{n}.
$$


In [45]:
# Generate random prime
e = number.getPrime(nsize + 1)
e_inv = pow(e, -1, (p-1)*(q-1))
# v is selected as integer
v_pprim = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate dot
R_vector = pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_4, m_4, n) * pow(R_5, m_5, n) * pow(R_6, m_6, n)

# Calculate commitment
commitment_vector = U * pow(S, v_pprim, n) * R_vector
commitment_vector_inv = pow(commitment_vector, -1, n)

# Calculate Q
Q = (Z * commitment_vector_inv) % n
# Calculate signature part A
A = pow(Q, e_inv, n)

print(f"Pre-signature is: A = {A}, e = {e}, v'' = {v_pprim}")

Pre-signature is: A = 6346188664443139069, e = 22715747490381789919, v'' = 2084761080170102586829415369587076326844573851810833884949501479953323642529969384916178563193928960775075430371102382137627136421895801774197793557773006614342783038048198022537041460667992959489689323136327514588416528753563164455359334353025084274089960532529057221105909505258124001477328575854894066257132488634336483585188292437983385492541315955193123663062990153190468859794720838900208215053776635432477404162609918013167352031746724819460176271502610930217816606027227839886124834268573704883726258151593839914013025950751223225943725566124676999743362911116246100355865999052632602426193854395157973406559229955236311975797620934769658947205171898917172578531254371917108432173198910479788506449609119598116801175883423139781848516806131091273668741396496541510549726607158948452444097613698110911606498728553749873376883632852296142571286074621409752622714601036542223732510904253618421838686319455023805058367489

Check
$$ Q = A^e $$

In [46]:
# verify
print('RHS is the same as LHS:', pow(A, e, n) == Q)

RHS is the same as LHS: True


### Issuer sends to User
Issuer sends pre-signature $(A,e,v'')$ to the User

## User complete signature

For signature completion user computes
$$v \leftarrow v'+v''$$

In [47]:
v = v_prim + v_pprim

User now know the signature $(A,e,v)$

User verify the signature $(A,e,v)$, using the CL verification algorithm:
1. Compute
$$
\hat{Z} \leftarrow A^{e} S^{v}(R_1^{m_1}R_2^{m_2}\cdots  R_i^{m_i})\pmod{n}.
$$

In [48]:
R_dot = pow(R_1, m_1, n) * R_vector
Z_hat = (pow(A, e, n) * pow(S, v, n) * R_dot) % n

Check
$$ Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) $$

In [49]:
print('RHS is the same as LHS:', Z_hat == Z)

RHS is the same as LHS: True


## Proof Generation


### Selective disclosure
Let A be the set of all attribute identifiers present in the credential schema
such that $A_r$ contains revealed attributes to the Verifier and $A_h$ contains unrevealed (hidden) attributes which are kept secret.


In [50]:
# public attributes
R_public = pow(R_4, m_4, n) * pow(R_5, m_5, n)
# private attributes
R_private = pow(R_1, m_1, n) * pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_6, m_6, n)

We remember that:
$$ Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) = 
A^e S^v \Bigg(\prod_{i \in A_r}{R_i^{m_i}} \cdot \prod_{i \in A_h}{R_i^{m_i}}\Bigg)
$$

Check
$$ Z \Bigg(\prod_{i \in A_r}{R_i^{m_i}}\Bigg)^{-1} = A^e S^v \Bigg(\prod_{i \in A_h}{R_i^{m_i}}\Bigg) $$

In [51]:
# generate commitment and encryption
R_public_inv = pow(R_public, -1, n)
# contains revealed attributes
attr_encryption_vector = Z * R_public_inv

# contains unrevealed attributes which are kept secret
attr_commitment_vector = pow(S, v, n) * R_private

# verify credentials
print('RHS is the same as LHS:', (pow(A, e, n) * attr_commitment_vector) % n == attr_encryption_vector % n)

RHS is the same as LHS: True


For each non-revealed attribute 

1. Generate random integer $\bar{m_1}, \bar{m_2}, \bar{m_3}$ for the secret attributes
2. Generates random integer $\bar{m_j}$ for each non-revealed attribute


In [52]:
m1_bar = number.getRandomNBitInteger(nsize)
m2_bar = number.getRandomNBitInteger(nsize)
m3_bar = number.getRandomNBitInteger(nsize)
m6_bar = number.getRandomNBitInteger(nsize)


Rbar_private = pow(R_1, m1_bar, n) * pow(R_2, m2_bar, n) * pow(R_3, m3_bar, n) * pow(R_6, m6_bar, n)


1. Generate random integer $r$
2. Then computes 
$$
A' \leftarrow A S^{r}\pmod{n}
$$
4. Then computes
$$
v' \leftarrow v - e\cdot r
$$


In [53]:
# Choose random bit r
r = number.getRandomNBitInteger(nsize + 1)

# Calculate
A_prime = A * pow(S, r, n)
v_prime = v - e*r

We remember that:

$$
Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) = A'^{e} S^{v'} \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) 
$$

Check
$$
A^e S^v = A'^{e} S^{v'}
$$
$$
A' S^{-r} = A \pmod{n}
$$

In [54]:
# we know that

print('RHS is the same as LHS:', (pow(A_prime, e, n) * pow(S, v_prime, n) % n * R_dot) % n == Z)
print('RHS is the same as LHS:', (pow(A_prime, e, n) * pow(S, v_prime, n) % n == pow(A, e, n) * pow(S, v, n) % n))
print('RHS is the same as LHS:', ((A_prime * pow(S, -r, n) % n == A % n)))

RHS is the same as LHS: True
RHS is the same as LHS: True
RHS is the same as LHS: True


1. Generate random integer $\beta$
2. Compute 
$$e' \leftarrow e - \beta$$

In [55]:
beta = number.getRandomNBitInteger(bitsize + 1)
e_prime = e - beta

print(f"Randomized signature is:\nA = {A_prime},\ne' = {e_prime},\nv' = {v_prime}")




Randomized signature is:
A = 52937058134205530488960810483642028319,
e' = 22715747483946234067,
v' = 39916780965271947323547114488799073236195712749794288349049820021088920341571476948571248791710537236356331894953933889697293729085409362206935725734902543545716269561790594699124815014468143116603612490889189004790370703605769187627117308911265835049017334804759483153683980824357294968334921528371612739686211403364616436199188144653838900291324133277964752771958345920062266846040933569645724082250610327393030522902876986736330112707329437573922679392893866782839070185282408423808507670235865237479861600821354996109539579208851187312318117018214698752620565671469752125934250426891074621925473319180279993095971995987484909402172721227022062913040945291587800398898973640858605626555167071761490592621528859198298740442930664932675055522013027859617013539103466649708613002694721999368320752518584796018161774181897833779054817752022127371211755178063573551304773616798526016911301504674183084

1. Generate random integer number $\bar{e}$
2. Generate random integer number $\bar{v}$
3. Compute
$$ T \leftarrow A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $$

In [56]:
e_bar = number.getRandomNBitInteger(nsize + 1)
v_bar = number.getRandomNBitInteger(msize*nsize+1)
T = (pow(A_prime, e_bar, n) * pow(S, v_bar, n) * Rbar_private) % n

### Hashing

Compute 
$$
c\leftarrow H(\mathcal{T},\mathcal{C},n_1)
$$

In [57]:
attr = str([T, Q, A, e, v, A_prime])
c_hash = hashlib.sha256(str(attr).encode('utf-8')).hexdigest()
c = int(c_hash, base=32)
print(f"Proof is: T = {T}, e_ = {e_prime},\nc = {c}")

Proof is: T = 2198160649117614273, e_ = 22715747483946234067,
c = 678598696908388092709236499864197236314747815112358918259807805632028283895893686444405505207296


Compute
\begin{align*}
\hat{v} \leftarrow \bar{v} + Dv' \\
\hat{e} \leftarrow \bar{e} + De'
\end{align*}

In [58]:
e_hat = e_bar + c*e_prime
v_hat = v_bar + c*v_prime

For each none revealed attribute, $j\in A_{h}$, compute
$$ \hat{m_j} \leftarrow \bar{m_j} + Dm_j $$


In [59]:
m1_hat = m1_bar + c*m_1
m2_hat = m2_bar + c*m_2
m3_hat = m3_bar + c*m_3
m6_hat = m6_bar + c*m_6

Rhat_private = pow(R_1, m1_hat, n) * pow(R_2, m2_hat, n) * pow(R_3, m3_hat, n) * pow(R_6, m6_hat, n)

We remember that:

\begin{equation}

\Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \\

= {\Bigg(\prod_{i \in A_r}{R_i^{Dm_i}}} \cdot \prod_{i \in A_h}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}} \\

= \Bigg(\prod_{i \in A}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}} \pmod{n} \\
\end{equation}

Lets check

\begin{equation}

\Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \\

= \Bigg(\prod_{i \in A}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}} \pmod{n} \\
\end{equation}


In [60]:
Rc_public = pow(R_4, m_4*c, n) * pow(R_5, m_5*c, n)
Rc_private = pow(R_1, m_1*c, n) * pow(R_2, m_2*c, n) * pow(R_3, m_3*c, n) * pow(R_6, m_6*c, n)
Rc_dot = Rc_private * Rc_public

print('RHS is the same as LHS:', (Rc_public * Rhat_private % n == Rc_dot * Rbar_private % n))

RHS is the same as LHS: True


## Proof Verification

Calculate

$$
\hat{T} \leftarrow \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg)
= Z^{-D} \cdot A'^{D\beta} \cdot A'^{\hat{e}} \cdot S^{\hat{v}} \cdot \Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n}
$$

In [61]:
T_hat = (pow(Z, -c, n) * pow(A_prime, c*beta, n) * pow(A_prime, e_hat, n) * pow(S, v_hat, n) * Rc_public * Rhat_private) % n

# we know that
print('RHS is the same as LHS:', (T_hat) == T)

RHS is the same as LHS: True


Compute
$$
\hat{Q} \leftarrow A'^{e} S^{-er} \pmod{n}
$$

In [62]:
Q_hat = pow(A_prime, e, n) * pow(S, -e*r, n) % n

Compute 
$$
\widehat{c} \leftarrow H(\hat{\mathcal{T}},\mathcal{C},n_1);
$$

In [63]:


attr_hat = str([T_hat, Q_hat, A, e, v, A_prime])
c_hash_hat = hashlib.sha256(str(attr_hat).encode('utf-8')).hexdigest()
c_hat = int(c_hash_hat, base=32)

print('RHS is the same as LHS:', (c_hat) == c)

RHS is the same as LHS: True


## Equality proof
This zero knowledge proof is in fact just an equality proof that does not reveal the attribute’s value.

In [64]:
assert([T, c] == [T_hat, c_hat])